# `01` Import Necessary Libraries

## `i` Default Libraries

In [1]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [16]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162993 sha256=107d838d802604222a8e7258bd7635ef67d4bc14d934de043ddcf6778b9afc47
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [22]:
import surprise

In [18]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [47]:
data = pd.read_csv("/content/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [4]:
utility_matrix = data.pivot(index='userID', columns='songID', values='rating')
utility_matrix.fillna(0, inplace=True)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [5]:
K = 3
U = np.random.rand(utility_matrix.shape[0], K)
M = np.random.rand(K, utility_matrix.shape[1])

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [6]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i, j] != 0: # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i, j] - np.dot(U[i, :], M[:, j]) # Calculate the error in prediction
                    for k in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] += lr * (2 * eij * M[k, j] - beta * U[i, k])
                        M[k, j] += lr * (2 * eij * U[i, k] - beta * M[k, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumulate the errors
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i, j] != 0: # Only proceed if the current interaction (i, j) is not missing
                    # since we only calculate the error for interactions having a ground truth value

                    first_part = (R[i, j] - np.dot(U[i, :], M[:, j])) ** 2 # calculate the first part of the error
                    second_part = 0 # Initialize a variable to accumulate the second part of the error

                    for k in range(K): # Loop over each latent features dimension
                        second_part += (beta/2) * (U[i, k]**2 + M[k, j]**2)

                    e += first_part + second_part # accumulate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last - e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M

## `iii` Use the Function to to Optimize the $U$ and $V$

In [14]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=100, lr=0.01, beta=0.01)

Epoch 1/100: Total Error = 10626.889477883282
Epoch 2/100: Total Error = 10578.176621114255
Epoch 3/100: Total Error = 10530.484829451923
Epoch 4/100: Total Error = 10483.78706694946
Epoch 5/100: Total Error = 10438.056584787468
Epoch 6/100: Total Error = 10393.266808110364
Epoch 7/100: Total Error = 10349.391298022998
Epoch 8/100: Total Error = 10306.403770600957
Epoch 9/100: Total Error = 10264.278156304292
Epoch 10/100: Total Error = 10222.988685157881
Epoch 11/100: Total Error = 10182.509985287104
Epoch 12/100: Total Error = 10142.817184674395
Epoch 13/100: Total Error = 10103.886008236655
Epoch 14/100: Total Error = 10065.69286440893
Epoch 15/100: Total Error = 10028.21491729409
Epoch 16/100: Total Error = 9991.430142078709
Epoch 17/100: Total Error = 9955.317362786594
Epoch 18/100: Total Error = 9919.85627256652
Epoch 19/100: Total Error = 9885.027437566636
Epoch 20/100: Total Error = 9850.812286086326
Epoch 21/100: Total Error = 9817.193085116072
Epoch 22/100: Total Error = 9784

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [15]:
# Predict ratings for user 199988
user_index = utility_matrix.index.get_loc(199988)
predicted_ratings = np.dot(U[user_index, :], M)

# Filter out songs already rated by user 199988
rated_songs = utility_matrix.iloc[user_index]
predicted_ratings[rated_songs != 0] = -np.inf

# get the indices of the top-K recommended songs
top_indices = np.argsort(predicted_ratings)[::-1][:5]

print("Top 5 Recommended Items for User 199988:")
for i, index in enumerate(top_indices, 1):
    print(f"\t- Top {i} Song: {index} (Predicted Rating: {predicted_ratings[index]})")


Top 5 Recommended Items for User 199988:
	- Top 1 Song: 46 (Predicted Rating: 5.9507725270556096)
	- Top 2 Song: 2 (Predicted Rating: 5.927688591520528)
	- Top 3 Song: 31 (Predicted Rating: 5.624669168344187)
	- Top 4 Song: 39 (Predicted Rating: 5.5363210849101465)
	- Top 5 Song: 44 (Predicted Rating: 5.404035069438319)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [30]:
reader = Reader(rating_scale=(1, 5))

In [33]:
data = Dataset.load_from_df(data, reader)
trainset, testset = train_test_split(data, test_size=0.25)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [23]:
# Biased Model
biased_model = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)

In [25]:
# Non-Biased Model
non_biased_model = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=100, n_epochs=20, biased=False, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)

## `iii` Fit each Model on Training Data

In [34]:
# Biased Model
biased_model.fit(trainset)

In [35]:
# Non-Biased Model
non_biased_model.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [39]:
# Biased Model
biased_predictions = biased_model.test(testset)
surprise.accuracy.mse(biased_predictions, verbose=True)

MSE: 2.2343


2.2342547141777

In [40]:
# Non-Biased Model
non_biased_predictions = non_biased_model.test(testset)
surprise.accuracy.mse(non_biased_predictions, verbose=True)

MSE: 4.3086


4.308642201237896

## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [44]:
# Biased Model
uid = 199988
iid = data[data['userID'] != uid]['songID'].unique()
preds=[]
for i in iid:
    pred = biased_model.predict(uid, i).est
    preds.append(pred)

In [45]:
data = {
    'songID': iid,
    'predicted_rating': preds
}
predictions = pd.DataFrame(data)
predictions.head()

,songID,predicted_rating
0,90409,3.396171
1,91266,3.306865
2,8063,4.040834
3,24427,4.137676
4,105433,3.840982


In [46]:
song_predictions_sorted = predictions.sort_values(by='predicted_rating' ,ascending=False )
song_predictions_sorted.head(10)

,songID,predicted_rating
30,62954,5.000000
19,71582,4.810036
37,2726,4.738270
42,2263,4.580222
24,113954,4.546280
27,55240,4.507075
33,56660,4.506637
46,45026,4.423072
22,22763,4.339459
36,48731,4.339146


In [48]:
# Non-Biased Model
uid = 199988
iid = data[data['userID'] != uid]['songID'].unique()
preds=[]
for i in iid:
    pred = non_biased_model.predict(uid, i).est
    preds.append(pred)

In [49]:
data = {
    'songID': iid,
    'predicted_rating': preds
}
predictions = pd.DataFrame(data)
predictions.head()

,songID,predicted_rating
0,90409,1.000000
1,91266,1.734647
2,8063,1.000000
3,24427,1.000000
4,105433,1.000000


In [50]:
song_predictions_sorted = predictions.sort_values(by='predicted_rating' ,ascending=False )
song_predictions_sorted.head(10)

,songID,predicted_rating
33,56660,4.706800
37,2726,4.699024
32,40712,2.909263
52,123176,2.380303
1,91266,1.734647
17,112023,1.601669
49,94604,1.572141
18,60465,1.553611
48,25182,1.542451
44,42781,1.309167
